In [5]:

import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
import tensorflow as tf
# Define input image dimensions
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

# Given input of noise (latent) vector, the Generator produces an image.
def build_generator():
    noise_shape = 100  # 1D array of size 100 (latent vector / noise)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1024, input_dim=noise_shape, activation='relu'))
    model.add(tf.keras.layers.Dense(784, activation='tanh'))
    model.add(tf.keras.layers.Reshape((28, 28, 1)))
    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)  # Generated image
    return Model(noise, img)

# Given an input image, the Discriminator outputs the likelihood of the image being real.
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=img_shape))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)

def train(epochs, batch_size=128, save_interval=50):
    save1 = ''
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3) 
    half_batch = int(batch_size / 2)
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
        noise = np.random.normal(0, 1, (batch_size, 100)) 
        valid_y = np.array([1] * batch_size) 

        g_loss = combined.train_on_batch(noise, valid_y)
        from IPython.display import clear_output
        clear_output(wait=True)
        print(save1)
        print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
        if epoch %100 ==0:
            save1 += ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]\n" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            save_imgs(epoch)
        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            generator.save(f'models/generator_{epoch}_MNIST_SIMPLES.h5')

# When the specific sample_interval is hit, we call the sample_image function.
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images_simples/s_mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view


# Let us also define our optimizer for easy use later on.
optimizer = Adam(0.0002, 0.5)  # Learning rate and momentum.

# Build and compile the discriminator first.
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

# Build and compile our Generator
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# This builds the Generator and defines the input noise.
z = Input(shape=(100,))  
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer='adam')  
train(epochs=100000, batch_size=32, save_interval=1000)

generator.save('generator_model.h5') 

0 [D loss: 0.492279, acc.: 65.62%] [G loss: 0.715556]
100 [D loss: 0.028949, acc.: 100.00%] [G loss: 3.122310]
200 [D loss: 0.055931, acc.: 100.00%] [G loss: 3.128373]
300 [D loss: 0.118371, acc.: 100.00%] [G loss: 2.127959]
400 [D loss: 0.088641, acc.: 100.00%] [G loss: 2.276102]
500 [D loss: 0.084362, acc.: 100.00%] [G loss: 2.346730]
600 [D loss: 0.041083, acc.: 100.00%] [G loss: 3.148790]
700 [D loss: 0.014633, acc.: 100.00%] [G loss: 4.988426]
800 [D loss: 0.012728, acc.: 100.00%] [G loss: 5.024119]
900 [D loss: 0.010827, acc.: 100.00%] [G loss: 6.042514]
1000 [D loss: 0.008043, acc.: 100.00%] [G loss: 5.738266]
1100 [D loss: 0.002181, acc.: 100.00%] [G loss: 6.532495]
1200 [D loss: 0.002320, acc.: 100.00%] [G loss: 7.307192]
1300 [D loss: 0.002535, acc.: 100.00%] [G loss: 6.243702]
1400 [D loss: 0.001325, acc.: 100.00%] [G loss: 6.500186]
1500 [D loss: 0.000443, acc.: 100.00%] [G loss: 7.634336]
1600 [D loss: 0.000437, acc.: 100.00%] [G loss: 7.919109]
1700 [D loss: 0.000663, acc